In [1]:
import numpy as np
import pandas as pd
import os
import re
from glob import glob

In [2]:
G49_raw_data = sorted(glob('./raw_data/G49/G*.txt'))

In [3]:
G49_raw_data_name = list(map(lambda x:x[15:],G49_raw_data))

In [4]:
i = 0
all_data = pd.DataFrame()
for file in G49_raw_data:
    data = pd.read_csv(file, sep = '\t')
    data['data_name'] = G49_raw_data_name[i]
    data.columns = data.columns.str.replace('  ','')
    data.columns = data.columns.str.replace('G49 -','G49-') #有些奇怪的間隔
    data['data_point'] = data.index
    all_data = pd.concat([data,all_data])
    all_data = all_data.drop([0]) #消除單位Deg.
    all_data.reset_index(drop = True)
    i = i + 1

C:\Users\AutoUser\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [5]:
all_data.columns

Index(['G49-PTC1', 'G49-PTC10', 'G49-PTC11', 'G49-PTC12', 'G49-PTC13',
       'G49-PTC14', 'G49-PTC15', 'G49-PTC16', 'G49-PTC17', 'G49-PTC18',
       'G49-PTC19', 'G49-PTC2', 'G49-PTC20', 'G49-PTC21', 'G49-PTC22',
       'G49-PTC23', 'G49-PTC25', 'G49-PTC26', 'G49-PTC27', 'G49-PTC28',
       'G49-PTC29', 'G49-PTC3', 'G49-PTC30', 'G49-PTC31', 'G49-PTC32',
       'G49-PTC33', 'G49-PTC34', 'G49-PTC35', 'G49-PTC36', 'G49-PTC37',
       'G49-PTC38', 'G49-PTC39', 'G49-PTC4', 'G49-PTC40', 'G49-PTC41',
       'G49-PTC42', 'G49-PTC43', 'G49-PTC44', 'G49-PTC45', 'G49-PTC46',
       'G49-PTC47', 'G49-PTC48', 'G49-PTC5', 'G49-PTC6', 'G49-PTC7',
       'G49-PTC8', 'G49-PTC9', 'Unnamed: 8', 'data_name', 'data_point'],
      dtype='object')

In [6]:
#drop 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8' 的columns
#rows出現 Deg的資料 欄位
for column in all_data.columns:
    if column[0] == 'U':
        all_data.drop([column],axis = 1,inplace = True)

In [7]:
all_data['data_name'] = all_data['data_name'].apply(lambda x:x[:-11]) #去掉 _Export.txt

In [8]:
all_data['data_name'].tail()

387    G49-AC(12X30)20160629-001
388    G49-AC(12X30)20160629-001
389    G49-AC(12X30)20160629-001
390    G49-AC(12X30)20160629-001
391    G49-AC(12X30)20160629-001
Name: data_name, dtype: object

In [9]:
all_data['recipes'] = all_data['data_name'].str.extract(r'(G\d+)',expand = False) #抓出所屬的Recipes
all_data['autoclave'] = all_data['data_name'].str.extract(r'(AC\(\d+x\d+\))',expand = False,flags=re.IGNORECASE) #用正則抓出 所屬的熱壓爐
all_data['date'] = all_data['data_name'].str.extract(r'(\d{7,8})',expand = False) #抓出日期
all_data['number'] = all_data['data_name'].str.extract(r'(00\d{1,3})',expand = False)#抓出所屬編號
all_data.drop(['data_name'],axis = 1,inplace = True)

In [10]:
all_data.columns = all_data.columns.str.replace('G49-','')

In [11]:
PTC_list = []
for column in all_data.columns:
    if column[0] == 'P' and column[-1] != 't':
        PTC_list.append(column)

In [12]:
all_data['PTC_count'] = len(PTC_list) - (all_data.shape[1] - all_data.count(axis = 1)) 
#每筆資料所收集溫度參數的個數

In [13]:
all_data['TemParameter'] = 0

In [14]:
G49_data = pd.DataFrame()

In [18]:
for PTC in PTC_list:
    data = all_data.loc[~all_data[PTC].isnull(),
             ['number', 'date', 'autoclave', 'recipes'
              ,'TemParameter', 'PTC_count','data_point',PTC]] #將各別 PTC 非0的數據挑出
    data['TemParameter'] = PTC  #建立 TemParameter 對應所屬來的PTC
    data['value'] = data[PTC]  # 賦予value值 統一
    data.drop([PTC],inplace=True,axis = 1)
    G49_data = pd.concat([G49_data,data])

In [19]:
G49_data.reset_index(drop=True, inplace=True)

In [20]:
G49_data['TemParameter'].value_counts()

PTC4     5502
PTC11    4699
PTC34    4337
PTC20    4294
PTC8     4231
PTC32    3965
PTC26    3920
PTC3     3902
PTC5     3555
PTC2     3420
PTC37    3185
PTC28    3113
PTC12    3084
PTC36    3083
PTC44    2752
PTC17    2737
PTC33    2725
PTC40    2722
PTC29    2708
PTC19    2459
PTC13    2446
PTC38    2330
PTC39    2010
PTC25    1982
PTC18    1939
PTC9     1881
PTC30    1651
PTC6     1648
PTC10    1601
PTC14    1572
PTC15    1569
PTC43    1557
PTC31    1524
PTC22    1218
PTC42    1196
PTC21    1187
PTC7     1182
PTC1     1171
PTC27    1156
PTC45    1152
PTC48     821
PTC46     787
PTC47     786
PTC41     779
PTC16     407
PTC35     391
PTC23     384
Name: TemParameter, dtype: int64

In [21]:
G49_data.to_csv('./training_data/G49_data.csv')

In [22]:
G49_data.head()

,number,date,autoclave,recipes,TemParameter,PTC_count,data_point,value
0,001,20160602,AC(12x30),G49,PTC1,8,1,85.1
1,001,20160602,AC(12x30),G49,PTC1,8,2,85.1
2,001,20160602,AC(12x30),G49,PTC1,8,3,85.1
3,001,20160602,AC(12x30),G49,PTC1,8,4,85.5
4,001,20160602,AC(12x30),G49,PTC1,8,5,85.6
